In [3]:
import numpy as np

In [10]:
def dists(rs, L):
    """
    Returns the difference vectors between all pairs of points in rs, given a periodic box of size L.
    
    Parameters
    ----------
    rs : (N,d) array_like
         Input vectors
    L : (d,) array_like, or scalar
    Size of the box
    
    Returns
    -------
    out : (N,N,d) ndarray
    Where out[i,j,:] is the difference vector between rs[i,:] and rs[j,:]
    """
    diffs = np.array([np.subtract.outer(rd, rd) for rd in rs.T]).T
    return np.remainder(diffs + L/2., L) - L/2.

In [11]:
N, L = 800, 2
d = np.random.uniform(high=L, size=(N,2))
dfs = dists(d, L)
drs = np.sqrt(np.sum(dfs**2, axis=-1))
np.sum(drs[np.triu_indices_from(drs)] < 1) / (N*(N+1)//2)

0.7848158551810237

In [12]:
print(d.shape, dfs.shape)

(800, 2) (800, 800, 2)


In [13]:
drs[np.triu_indices_from(drs)].shape

(320400,)

In [14]:
%%timeit -n 1 -r 1
N = 200
d = 3
L = 2.0
Nruns = 500

in_disk = np.zeros(Nruns)
for i in range(Nruns):
    rs = np.random.uniform(size=(N,d), high=(L*2))
    diffs = dists(rs, L)
    drs = np.sqrt(np.sum(diffs**2, axis=-1))
    #in_disk[i] = np.sum(drs[np.triu_indices_from(drs)] < 1)
    in_disk[i] = (np.sum(drs < 1) - N) / 2

1 loops, best of 1: 1.33 s per loop


In [44]:
def dists2(rs, L):
    """
    Returns the difference vectors between all pairs of points in rs, given a periodic box of size L.
    
    Parameters
    ----------
    rs : (N,d) array_like
         Input vectors
    L : (d,) array_like, or scalar
    Size of the box
    
    Returns
    -------
    out : (N,N,d) ndarray
    Where out[i,j,:] is the difference vector between rs[i,:] and rs[j,:]
    """
    N,d = np.shape(rs)
    diffs = np.zeros((N,N,d))
    for i,rd in enumerate(rs.T):
        np.subtract.outer(rd, rd, out=diffs[:,:,i])
    return np.remainder(diffs + L/2., L) - L/2.

In [16]:
%%timeit rs = np.random.uniform(size=(100,3), high=(4.0))
diffs = dists(rs, 2.0)

1000 loops, best of 3: 511 µs per loop


In [29]:
%%timeit rs = np.random.uniform(size=(100,3), high=(4.0))
diffs = dists2(rs, 2.0)

1000 loops, best of 3: 617 µs per loop


In [45]:
def dists3(rs, L):
    """
    Returns the difference vectors between all pairs of points in rs, given a periodic box of size L.
    
    Parameters
    ----------
    rs : (N,d) array_like
         Input vectors
    L : (d,) array_like, or scalar
    Size of the box
    
    Returns
    -------
    out : (N,d,N) ndarray
    Where out[i,:,j] is the difference vector between rs[i,:] and rs[j,:]
    """
    N,d = np.shape(rs)
    diffs = rs[..., np.newaxis] - rs.T[np.newaxis, ...]
    diffs = np.rollaxis(diffs, -1, -2)
    return np.remainder(diffs + L/2., L) - L/2.

In [46]:
%%timeit rs = np.random.uniform(size=(100,3), high=(4.0))
diffs = dists3(rs, 2.0)

1000 loops, best of 3: 614 µs per loop


In [48]:
np.allclose(dists2(d, 2.0), dists3(d, 2.0))

True

In [38]:
diffs = d[..., np.newaxis] - d.T[np.newaxis, ...]
diffs.shape
np.rollaxis?

In [24]:
rx = d[..., np.newaxis]

In [35]:
dists3(d, np.array((L, L)))

ValueError: operands could not be broadcast together with shapes (800,2,800) (2,) 

### Pairs of Particles

In [55]:
def pair_dists(rs1, rs2, L):
    """
    Returns the difference vectors between all pairs of points in rs1 and rs2, given a periodic box of size L.
    
    Parameters
    ----------
    rs1 : (N,d) array_like
         Input vectors
    rs2 : (N,d) array_like
         Input vectors
    L : (d,) array_like, or scalar
        Size of the box
    
    Returns
    -------
    out : (N,N,d) ndarray
    Where out[i,:,j] is the difference vector between rs1[i,:] and rs2[j,:]
    """
    N,d = np.shape(rs)
    diffs = rs1[..., np.newaxis] - rs2.T[np.newaxis, ...]
    diffs = np.rollaxis(diffs, -1, -2)
    return np.remainder(diffs + L/2., L) - L/2.

In [117]:
def pair_dist(rs1, rs2, L):
    """
    Returns the total distance between pairs of points in rs1 and rs2, given a 
    periodic box of size L. Ignores overall translation.
    
    Parameters
    ----------
    rs1 : (N,d) array_like
         Input vectors
    rs2 : (N,d) array_like
         Input vectors
    L : (d,) array_like, or scalar
        Size of the box
    
    Returns
    -------
    out : sqrt(sum(r_i - s_i - delta)**2), where "delta" would be the
        center-of-mass translation from r_i to s_i.
    """
    dists1 = (rs1[..., np.newaxis] - rs1.T[np.newaxis, ...])
    dists2 = (rs2[..., np.newaxis] - rs2.T[np.newaxis, ...])
    rij_minus_sij = np.rollaxis(dists1 - dists2, -1, -2)
    rij_minus_sij = ((rij_minus_sij + L/2.) % L) - L/2.
    dist_sqs = np.triu(np.sum(rij_minus_sij**2, axis=-1))
    return np.sqrt(np.sum(dist_sqs) / len(rs1))

In [118]:
L = 4
N = 20
d = 3
jitter = 0.1
rs = np.random.uniform(0.0, L, size=(N,d))
diffs = np.random.uniform(-jitter, jitter, size=(N,d))
diffs -= np.mean(diffs, axis=0)
rs2 = (rs + diffs) % L

In [119]:
# Actual distance
diffs = np.sum(pair_dists(rs, rs2, L)**2, axis=-1)
np.sqrt(np.trace(diffs))

0.46390270838165865

In [120]:
# Distance minus translations
pair_dist(rs, rs2, L)

0.46390270838165865